In [2499]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Значения признаков:

client_id - идентификатор клиента
education - уровень образования
sex - пол заемщика
age - возраст заемщика
car - флаг наличия автомобиля
car_type - флаг автомобиля иномарки
decline_app_cnt - количество отказанных прошлых заявок
good_work - флаг наличия “хорошей” работы
bki_request_cnt - количество запросов в БКИ
home_address - категоризатор домашнего адреса
work_address - категоризатор рабочего адреса
income - доход заемщика
foreign_passport - наличие загранпаспорта
sna - связь заемщика с клиентами банка
first_time - давность наличия информации о заемщике
score_bki - скоринговый балл по данным из БКИ
region_rating - рейтинг региона
app_date - дата подачи заявки
default - флаг дефолта по кредиту

In [2500]:
#вначале подгрузим все необходимые нам в дальнейшем библиотеки

from pandas import Series

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

from imblearn.under_sampling import RandomUnderSampler


In [2501]:
#дополнительные настройки

import warnings 
warnings.filterwarnings("ignore")

In [2502]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [2503]:
sample_submission.shape

In [2504]:
df_test.shape

In [2505]:
df_train.info()

В данных видим 6 категориальных признаков, 12 числовых, а также один целевой параметр внутри обучающей выборки - default. 

In [2506]:
df_train.head(5)

In [2507]:
df_test.info()

In [2508]:
sample_submission.head(5)

In [2509]:
sample_submission.info()

In [2510]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения default, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

**Предварительная работа с данными**

In [2511]:
data

In [2512]:
data.nunique(dropna=False)

Наибольшее количество уникальных значений у 'client_id' и 'score_bki'. Можно ожидать сильного влияния 'score_bki' на целевую переменную.

Видим, что в сравнении с тем, что мы выполняли на платформе, к этому датасету прибались новые данные. 

In [2513]:
data.drop(['client_id','app_date'], axis = 1, inplace=True)

In [2514]:
#поработаем с app_date

In [2515]:
#посмотрим на пропуски

data.isnull().sum()

In [2516]:
#заполняем самым частым значением и после проверяем

data['education'].fillna(data.education.mode()[0],inplace=True)

In [2517]:
data.education.value_counts(dropna=False)

In [2518]:
#посмотрим на все параметры по очереди, при числовых - смотрим на графики и логарифмируем, если нужно

data.sex.value_counts()




In [2519]:
data.age.hist()

In [2520]:
#логарифмируем, чтобы попытаться привести к виду нормального распределения

data.age.apply(lambda x: np.log(x)).hist()

In [2521]:
data.car.value_counts()

In [2522]:
data.car_type.value_counts()

In [2523]:
data.decline_app_cnt.hist()

In [2524]:
data.good_work.value_counts()

In [2525]:
data.score_bki.hist()

In [2526]:
data.bki_request_cnt.hist()

In [2527]:
data.region_rating.hist()

этот параметр можно отнести как к категориальным, так и к числовым

In [2528]:
data.work_address.value_counts()

In [2529]:
data.income.hist()

In [2530]:
#логарифмируем

data.income.apply(lambda x: np.log(x)).hist()

In [2531]:
data.sna.value_counts()

In [2532]:
data.first_time.value_counts()

In [2533]:
data.foreign_passport.value_counts()

Теперь посмотрим на данные в обучающей выборке

In [2534]:
df_train.default.value_counts()

Отмечаем диспропорцию

In [2535]:
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
cat_cols = ['education', 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

**Работа с числовыми переменными**

In [2536]:

sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1,annot=True)

Корреляции между числовыми переменными слабые, что хорошо, можно использовать все.

In [2537]:
#что насчет значимости?

imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Самая значимая переменная - score_bki

**Работа с бинарными и категориальными переменными**

Выше мы уже видели, что некоторые  переменные обозначены буквами, поэтому поменяем их на числовые выражения

In [2538]:
label_encoder = LabelEncoder()

for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    print(column, dict(enumerate(label_encoder.classes_)))

Определяем значимость бинарных и категориальных переменных. 

In [2539]:
label_encoder = LabelEncoder()

data['education'] = pd.Series(label_encoder.fit_transform(data['education']))
print(dict(enumerate(label_encoder.classes_)))

In [2540]:
imp_cat = Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'], discrete_features=True), index=bin_cols + cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

Самый значимый параметр - sna

In [2541]:
data

**Подготовка данных**

In [2542]:
data = pd.get_dummies(data, columns=cat_cols, dummy_na=False)

In [2543]:
#cтандартизируем числовые признаки

data[num_cols] = pd.DataFrame(StandardScaler().fit_transform(data[num_cols]), columns=num_cols)

**Построение модели**

In [2544]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)

In [2545]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
from sklearn.model_selection import train_test_split

# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2546]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

**Гиперпараметры**

In [2547]:
#logreg = LogisticRegression(random_state=random_seed)

#random_seed = 42
#C = np.logspace(-2, 1, 10)
iter_ = 100
#epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

#gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
#gridsearch.fit(х_train, y_train)
#model = gridsearch.best_estimator_

#печатаем параметры

#best_parameters = model.get_params()
#for param_name in sorted(best_parameters.keys()):
        #print('\t%s: %r' % (param_name, best_parameters[param_name]))

Здесь получилось очень долгое выполнение кода, поэтому мы перед ними добавили #. Перейдем сразу к полученным для модели данным:

C: 1.0

class_weight: 'balanced'

dual: False

fit_intercept: True

intercept_scaling: 1

l1_ratio: None

max_iter: 50

multi_class: 'auto'

n_jobs: None

penalty: 'l1'

random_state: 42

solver: 'liblinear'

tol: 0.001

verbose: 0

warm_start: False



**Обучение модели при помощи логистической регрессии при учете гиперпараметров**

In [2548]:
logreg = LogisticRegression(tol=0.001,solver='liblinear', max_iter=50, class_weight='balanced', random_state=42)
logreg.fit(х_train, y_train)
y_pred = model.predict(х_test)

In [2549]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test, y_pred)
print(classification_report)

In [2550]:
#Нарисуем кривую Roc и вычислим AUC

fpr, tpr, thresholds = roc_curve(y_train,logreg.predict_proba(X_train).T[1])
roc_auc = roc_auc_score(y_train,logreg.predict_proba(X_train).T[1])   
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.title('Receiver Operating Characteristic', fontsize=15)
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.legend(fontsize=15)

In [2551]:
#выведем матрицу ошибок

print('confusion_matrix:')
print(confusion_matrix(y_test,y_pred))

In [2552]:
#попробуем применять метод undersampling и снова посмотреть на выход

from imblearn.under_sampling import RandomUnderSampler

random_seed = 42

undersampling = RandomUnderSampler(random_state=42)

X_uns, y_uns = undersampling.fit_resample(X, y)

X_train_uns, X_test_uns, y_train_uns, y_test_uns = train_test_split(X_uns, y_uns, test_size=0.10, shuffle = True, random_state=random_seed)

logreg_uns = LogisticRegression(solver='saga', max_iter=1000, random_state=random_seed)

logreg_uns.fit(X_train_uns, y_train_uns)

y_pred_uns = logreg_uns.predict(X_test_uns)

In [2553]:
fpr, tpr, thresholds = roc_curve(y_train_uns,logreg_uns.predict_proba(X_train_uns).T[1])
roc_auc = roc_auc_score(y_train_uns,logreg_uns.predict_proba(X_train_uns).T[1])   
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.4f}')
plt.title('Receiver Operating Characteristic', fontsize=15)
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.legend(fontsize=15)

In [2554]:
print('confusion_matrix:')
print(confusion_matrix(y_test_uns,y_pred_uns))

**Обучение модели**

In [2555]:
# если качество нас устраивает, обучаем финальную модель на всех обучающих данных

logreg_final = LogisticRegression(tol=0.001,solver='liblinear', max_iter=50, class_weight='balanced', random_state=42)
logreg_final.fit(X, y)

In [2556]:
predict_submission = logreg_uns.predict(test_data)

In [2557]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [2558]:
sample_submission.describe()

In [2559]:
!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'